In [ ]:
import yfinance as yf
import pandas as pd
import csv
import numpy as np

# Stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten list of stocks
stock_symbols = [symbol for sector in us_stock_universe.values() for symbol in sector]

# Date range
start_date = "2021-03-20"
end_date = "2022-03-20"

# Download weekly data
data = yf.download(stock_symbols, start=start_date, end=end_date, interval='1wk')
close_prices = data['Close'].reset_index()

# Melt the data to long format
df = close_prices.melt(id_vars='Date', var_name='Stock', value_name='Closing Price')

# Format the date
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# Reorder columns: Stock, Date, Closing Price
df = df[['Stock', 'Date', 'Closing Price']]

# Save to CSV
df.to_csv('Backtest_training_prices.csv', index=False)


In [ ]:
import pandas as pd

# Read the data
df = pd.read_csv("Backtest_training_prices.csv")

# Strip spaces and convert Date Fetched column to datetime (Handles both date and timestamp formats)
df["Date"] = pd.to_datetime(df["Date"].str.strip(), errors='coerce')

# Check for any rows where Date Fetched couldn't be converted
if df["Date"].isna().sum() > 0:
    print("Warning: Some dates could not be parsed. Check for inconsistencies.")

# Save cleaned data (optional)
df.to_csv("Backtest_training_prices.csv", index=False)

# Display the first few rows
print(df.head())

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime, timedelta

# Read the Stock Prices Data
df = pd.read_csv("Backtest_training_prices.csv", dtype={"Date": str})  # Read as string first
df["Date"] = df["Date"].str.strip()  # Remove any extra spaces
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")  # Convert to datetime

# Sort by Date (to ensure proper time series order)
df = df.sort_values(by=["Stock", "Date"]).reset_index(drop=True)

# Get Unique Stocks
unique_stocks = df["Stock"].unique()

# Store Predictions
predictions = []

today = (
    [datetime(2022, i, 21) for i in range(3, 13)] +
    [datetime(2023, i, 21) for i in range(1, 13)] +
    [datetime(2024, i, 21) for i in range(1, 4)]
)

# Iterate Over Each Stock to Apply SARIMA Model
for i in range(len(today)):
    for stock in unique_stocks:
        stock_data = df[df["Stock"] == stock]
        stock_data = stock_data.set_index("Date")["Closing Price"].dropna()
    
        # Check if there is enough data
        if len(stock_data) < 10:
            print(f"Not enough data for {stock}, skipping...")
            continue
    
        # Fit SARIMA Model (Seasonal Order assumes weekly seasonality)
        try:
            model = SARIMAX(stock_data, order=(1,1,1), seasonal_order=(1,1,1,52), enforce_stationarity=False, enforce_invertibility=False)
            results = model.fit(disp=False)
        
            # Forecast Today's Price
            forecast = results.forecast(steps=1)
            predicted_price = forecast.iloc[0]
            predictions.append({"Stock": stock, "Predicted Price ($)": round(predicted_price, 2), "Date Predicted": today[i]})
        
            print(f" Predicted Price for {stock} on {today[i]}: ${predicted_price:.2f}")
        except Exception as e:
            print(f" Error processing {stock}: {e}")

# Convert Predictions to DataFrame
pred_df = pd.DataFrame(predictions)
print(pred_df)

# Save Predictions to CSV (Optional)
pred_df.to_csv("Backtest_price_predicts.csv", index=False)

 Predicted Price for HD on 2022-03-21 00:00:00: $499.32


c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for INTC on 2022-03-21 00:00:00: $83.68


c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for JNJ on 2022-03-21 00:00:00: $258.16


...

 Predicted Price for WFC on 2024-03-21 00:00:00: $71.53


c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for WMT on 2024-03-21 00:00:00: $74.69


c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for XOM on 2024-03-21 00:00:00: $103.77
     Stock  Predicted Price ($) Date Predicted
0     AAPL               240.41     2022-03-21
1      AMD               165.07     2022-03-21
2     AMZN               262.98     2022-03-21
3     AVGO                86.19     2022-03-21
4       BA               356.08     2022-03-21
...    ...                  ...            ...
1245   UNH               720.96     2024-03-21
1246    VZ                72.34     2024-03-21
1247   WFC                71.53     2024-03-21
1248   WMT                74.69     2024-03-21
1249   XOM               103.77     2024-03-21

[1250 rows x 3 columns]


In [4]:
# Define the stock order from us_stock_universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten the stock order list
stock_order = [stock for sector in us_stock_universe.values() for stock in sector]

# Load the CSV file
df = pd.read_csv("Backtest_price_predicts.csv")

# Ensure the stock column is treated as a category with the correct order
# Get number of rows and number of 50-stock groups
n = len(df)
group_size = 50
num_groups = (n + group_size - 1) // group_size  # ceiling division

sorted_df_list = []

for i in range(num_groups):
    start_idx = i * group_size
    end_idx = min(start_idx + group_size, n)
    
    group_df = df.iloc[start_idx:end_idx].copy()
    group_df["Stock"] = pd.Categorical(group_df["Stock"], categories=stock_order, ordered=True)
    group_df = group_df.sort_values("Stock")
    
    sorted_df_list.append(group_df)

# Concatenate all sorted groups
df_sorted = pd.concat(sorted_df_list).reset_index(drop=True)

# Sort the dataframe by the defined stock order

# Save the sorted dataframe
df_sorted.to_csv("Backtest_price_predicts.csv", index=False)

print(" Data successfully saved in Backtest_price_predicts.csv in the correct sector and stock order.")

 Data successfully saved in Backtest_price_predicts.csv in the correct sector and stock order.


In [5]:
Worth=(pd.read_csv("backtesting_data.csv")["Price"])-(pd.read_csv("Backtest_price_predicts.csv")["Predicted Price ($)"])
# Load existing CSV
df = pd.read_csv("backtesting_data.csv")

# Add a new column
df["Worth"] = Worth

# Save it back to the same file
df.to_csv("backtesting_data.csv", index=False)

In [6]:
import csv

def fetch_row(file_name, row_index):
    with open(file_name, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for idx, row in enumerate(reader):
            if idx == row_index:
                try:
                    # Extract columns 2 to 9 (8 features), convert to float
                    return [float(x.strip()) if x.strip() else 0.0 for x in row[2:10]]
                except ValueError as e:
                    print(f"ValueError at row {row_index}: {e}")
                    return [0.0] * 8  # Fallback to zeros if error
    return None


In [7]:
import ast

x_test=np.zeros((50,8))
ratio=np.zeros(1250)
w=pd.read_csv("Weight_Bias.csv")["Weights"].to_numpy()

b_str = pd.read_csv("Weight_Bias.csv")["Bias"].iloc[0]
b = ast.literal_eval(b_str)[0]  # Extract float from list

for j in range(25):
    for i in range(50):
        x_test[i]=fetch_row("backtesting_data.csv",50*j+i)
        ratio[50*j+i]=np.dot(x_test[i],w)+b

mean = np.mean(x_test, axis=0)
std = np.std(x_test, axis=0)
std[std == 0] = 1  # Prevent division by zero
x_test = (x_test - mean) / std

final_result={
    "Stock": pd.read_csv("backtesting_data.csv")["Stock"],
    "Price":pd.read_csv("backtesting_data.csv")["Price"],
    "Weights":ratio
}
df=pd.DataFrame(final_result)
df.to_csv("Backtesting_results.csv",index=False)

In [8]:
weights=pd.read_csv("Backtesting_results.csv")["Weights"]
num_row=len(weights)
weight_sum=[0 for i in range(25)]
chosen_index=[[-1 for i in range(10)]for j in range(25)]

In [9]:
num_groups = 25
group_size = 50

# Initialize containers
chosen_index = [[-1 for _ in range(10)] for _ in range(num_groups)]
weight_sum = [0 for _ in range(num_groups)]

weights = pd.read_csv("Backtesting_results.csv")["Weights"].copy()  # make sure it's mutable

# Step 1: Select top 10 per group
for k in range(num_groups):  # loop over 25 groups
    group_indices = list(range(group_size))
    group_global_indices = [group_size * k + i for i in group_indices]

    # Get weights for current group
    current_group_weights = [weights[idx] for idx in group_global_indices]

    # Select top 10 local indices based on weight
    top_10_local_indices = sorted(range(group_size), key=lambda i: current_group_weights[i], reverse=True)[:10]

    # Save and compute weight sum
    for j, local_i in enumerate(top_10_local_indices):
        global_i = group_size * k + local_i
        chosen_index[k][j] = local_i
        weight_sum[k] += weights[global_i]

# Step 2: Normalize selected weights as percentages
for k in range(num_groups):
    for local_i in chosen_index[k]:
        global_i = group_size * k + local_i
        weights[global_i] = weights[global_i] * 100 / weight_sum[k]


In [10]:
import pandas as pd

df_raw = pd.read_csv("Backtesting_results.csv")
weights = df_raw["Weights"]
stock_col = df_raw["Stock"]
price_col = df_raw["Price"]

final_stock = []
final_price = []
final_weights = []

for k in range(25):
    for i in chosen_index[k]:  # i is local index within the 50-stock group
        global_index = 50 * k + i
        final_stock.append(stock_col.iloc[global_index])
        final_price.append(price_col.iloc[global_index])
        normalized_weight = weights[global_index] * 100 / weight_sum[k]
        final_weights.append(normalized_weight)

# Create and save final DataFrame
data = {
    "Stock": final_stock,
    "Price": final_price,
    "Weights": final_weights
}

df = pd.DataFrame(data)
df.to_csv("Backtest_final_investment.csv", index=False)


In [11]:
import pandas as pd

# Example DataFrame (assumed already loaded with 1250 rows)
# df = pd.read_csv("your_file.csv")

# Create an empty DataFrame to store sorted groups
sorted_df = pd.DataFrame()

# Loop through DataFrame in chunks of 10 rows
for i in range(0, len(df), 10):
    group = df.iloc[i:i+10]                                 # Select 10-row group
    sorted_group = group.sort_values(by="Weights", ascending=False)  # Sort descending
    sorted_df = pd.concat([sorted_df, sorted_group], ignore_index=True)

# Optional: Reset index for clean indexing
sorted_df = sorted_df.reset_index(drop=True)

sorted_df.to_csv("Backtest_final_investment.csv", index=False)

# Output result
print(sorted_df)


    Stock   Price    Weights
0    COST  269.19  18.533784
1     NOC  170.29  14.268745
2     UNH  370.32  12.028642
3      GS  143.25  11.368190
4      HD  168.26  11.000314
..    ...     ...        ...
245  TSLA  205.86   7.192671
246    GS  308.08   6.095099
247    HD  288.47   5.893550
248   LLY  255.40   3.758112
249    GD  147.03   3.268632

[250 rows x 3 columns]


In [12]:
def Price_update(k):
    # Load CSV file
    Price = pd.read_csv("Backtest_final_investment.csv")["Price"]
    new_prices=[Price[i] for i in range(k,k+10)].copy()
    
    return new_prices

In [13]:
def Weight_update(k):
    # Load CSV file
    Weigh = pd.read_csv("Backtest_final_investment.csv")["Weights"]
    new_weights=[Weigh[i] for i in range(k,k+10)]
    
    return new_weights

In [14]:
def Stock_update(k):
    # Load CSV file
    stock_n = pd.read_csv("Backtest_final_investment.csv")["Stock"]
    new_stocks=[stock_n[i] for i in range(k,k+10)]
    
    return new_stocks

In [15]:
wallet=10000.00
stock_old_price=[0 for q in range(10)]
returns=0
quantity=[[0 for q in range(10)]  for p in range(25)]

##### Finding quantity of each stock to buy and then buying and selling them on 3-month basis

In [16]:
for k in range(25):
    stock_weights=np.array(Weight_update(k))
    stock_buy_price=np.array(Price_update(k))
    stocks_name=np.array(Stock_update(k))
    stock_quant = (wallet * (stock_weights / 100)) // stock_buy_price
    best_choice=stock_buy_price[0]
    best_choice_index=-1
    current_wallet=0.0 
    for i in range(len(stocks_name)):
        stock_quant[i]=int(stock_quant[i])
        current_wallet+=stock_quant[i]*stock_buy_price[i]
    for i in range(len(stocks_name)//2):
        if(stock_buy_price[i]<best_choice):
            best_choice=stock_buy_price[i]
            best_price_index=i

    current_wallet=wallet-current_wallet
    if(current_wallet//best_choice>0):
        stock_quant[best_price_index]+=int(current_wallet//best_choice)

    current_wallet=0
    for i in range(len(stocks_name)):
        stock_quant[i]=int(stock_quant[i])
        current_wallet+=stock_quant[i]*stock_buy_price[i]
    for i in range(best_choice_index,len(stocks_name)):
        if(stock_buy_price[i]<best_choice):
            best_choice=stock_buy_price[i]
            best_price_index=i

    current_wallet=wallet-current_wallet
    if(current_wallet//best_choice>0):
        stock_quant[best_price_index]+=int(current_wallet//best_choice)

    current_wallet-=(current_wallet//best_choice)*best_choice
    quantity[k]=stock_quant
    

### Trading

In [17]:
stock_old_price=np.array([])
stocks_name_old=([])
print(stock_old_price.size==0)
for k in range(25):
    map=[0 for q in range(10)]

    stock_weights=np.array(Weight_update(k))
    stock_buy_price=np.array(Price_update(k))
    stocks_name=np.array(Stock_update(k))

    if (stock_old_price.size==0):
        for j in range(10):
            if(stock_buy_price[j]>0):
                wallet-=quantity[k][j]*stock_buy_price[j]
    else:
        for i in range(len(stock_buy_price)):
            for j in range(len(stocks_name)):
                if stocks_name_old[i] == stocks_name[j] and stock_buy_price[j]>stock_old_price[i]:
                    diff = quantity[k-1][i] - quantity[k][j]
                    if diff > 0 and stock_buy_price[j]>0 and stock_old_price[i]>0:
                        wallet += diff * stock_old_price[i]
                    elif diff < 0 and stock_buy_price[j]>0 and stock_old_price[i]>0:
                        wallet -= diff * stock_buy_price[j]
                    elif diff < 0 and stock_buy_price[j]>0 and stock_old_price[i]<0:
                        wallet -= quantity[k][j] * stock_buy_price[j]
        if stock_old_price.size!=0:
            for j in range(len(stocks_name)):
                if(map[j]!=1) and stock_old_price[j]>0:
                    wallet+=stock_old_price[j]*quantity[k-1][j]
    stocks_name_old=stocks_name.copy()
    stock_old_price=stock_buy_price.copy()

True


In [18]:
returns=wallet-10000
print(f"Returns = {returns}\n Wallet = {wallet}")

Returns = 243616.84000000008
 Wallet = 253616.84000000008
